# Twitter Sentiment Analysis 

# ARTG Project - By Team Maple

**<font color="blue">Import libraries</font>**  

**<font color="red">Code:</font>**

In [84]:
import csv
import sys
!conda install --yes --prefix {sys.prefix} -c conda-forge tweepy
import tweepy

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\prakh\anaconda3

  added / updated specs:
    - tweepy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blinker-1.4                |             py_1          13 KB  conda-forge
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    oauthlib-3.0.1             |             py_0          82 KB  conda-forge
    pyjwt-1.7.1                |             py_0          17 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    requests-oauthlib-1.2.0    |             py_0          19 KB  conda-forge
    tweepy-3.9.0               |     pyh9f0ad1d_0          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages w

**<font color="blue">Step 1: Twitter crawl & downloading data</font>**  

**<font color="red">Code:</font>**

In [ ]:
consumer_key = 'kmflAhDlFLYvVLzUhUTv5Agbw'
consumer_secret = 'D8Giq3GMjIMaBQN0Qw63tsoHpajA7hNKy2ouo1XeqQP46SP38C'
access_token = '4266439228-U0ySetwuTNEtz3ZGdyPWKVrfIGKY866EMIrzNbN'
access_token_secret = 'CVynCeCX22NtK8iygTHGWjota2JhGTfPgk2CO1MUnPfBK'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, timeout=1000)

In [ ]:
# Open/create a file to append data to
csvFile = open('testgf.csv', 'a')

In [ ]:
# Used csv writer
csvWriter = csv.writer(csvFile)

In [ ]:
for tweet in tweepy.Cursor(api.search,
                           q="#blacklivesmatter",
                           # Data only available for last 7 days. We mined tweets for 4 days - 24,25,26,27 June
                           since="2020-06-24",
                           # Until 28 means  <28 i.e. <=27
                           until="2020-06-28",
                           lang="en",
                           count=100).items():
    # Write a row to the CSV file. We used encode UTF-8
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
    print(tweet.created_at, tweet.text)
csvFile.close()

**<font color="blue">Step 2: Preprocessing the raw dump. Cleaning the tweets for NLP</font>**  

**<font color="red">Code:</font>**

In [82]:
import sqlite3
import pandas as pd
#!conda install tweet-preprocessor
import preprocessor as p
#!conda install gensim
from gensim.parsing.preprocessing import remove_stopwords

In [69]:
# Read the raw tweets mined
raw_twts = pd.read_csv('testgf.csv', header=None)

In [70]:
# Name the columns
raw_twts.columns = ['datetime', 'tweet']

In [71]:
# Insert a new ID column
raw_twts.insert(loc=0, column='id', value=1)

In [72]:
# Assign unique vaues to id column for each row 
raw_twts["id"] = raw_twts.index + 1000000

In [61]:
raw_twts.head()

,id,datetime,tweet
0,1000000,6/27/2020 23:59,"b""RT @KashJackson2018: @RealCandaceO Please he..."
1,1000001,NaN,NaN
2,1000002,6/27/2020 23:59,"b""RT @AndyLunique: Hi. It's not over. Don't le..."
3,1000003,NaN,NaN
4,1000004,6/27/2020 23:59,"b""RT @AndyLunique: Hi. It's not over. Don't le..."


In [73]:
# Dropping duplicates and NaN from the dataframe
raw_twts = raw_twts.dropna()
raw_twts = raw_twts.drop_duplicates(subset='tweet', keep='first')

In [63]:
raw_twts.head()

,id,datetime,tweet
0,1000000,6/27/2020 23:59,"b""RT @KashJackson2018: @RealCandaceO Please he..."
2,1000002,6/27/2020 23:59,"b""RT @AndyLunique: Hi. It's not over. Don't le..."
6,1000006,6/27/2020 23:59,"b""RT @Selena_Adera: Stop treating our communit..."
8,1000008,6/27/2020 23:59,b'RT @DeAnna4Congress: Only 17% of the #BlackL...
10,1000010,6/27/2020 23:59,b'RT @Hoosiers1986: Rational people have alway...


In [105]:
raw_twts.to_csv ('raw_twts.csv', index = False, header=True)

In [91]:
raw_twts.head()

,id,datetime,tweet
0,1000000,6/27/2020 23:59,"b""RT @KashJackson2018: @RealCandaceO Please he..."
2,1000002,6/27/2020 23:59,"b""RT @AndyLunique: Hi. It's not over. Don't le..."
6,1000006,6/27/2020 23:59,"b""RT @Selena_Adera: Stop treating our communit..."
8,1000008,6/27/2020 23:59,b'RT @DeAnna4Congress: Only 17% of the #BlackL...
10,1000010,6/27/2020 23:59,b'RT @Hoosiers1986: Rational people have alway...


In [106]:
clean_twts = pd.read_csv('raw_twts.csv')
clean_twts.count()

id          98907
datetime    98907
tweet       98907
dtype: int64

In [107]:
clean_twts.head()

,id,datetime,tweet
0,1000000,6/27/2020 23:59,"b""RT @KashJackson2018: @RealCandaceO Please he..."
1,1000002,6/27/2020 23:59,"b""RT @AndyLunique: Hi. It's not over. Don't le..."
2,1000006,6/27/2020 23:59,"b""RT @Selena_Adera: Stop treating our communit..."
3,1000008,6/27/2020 23:59,b'RT @DeAnna4Congress: Only 17% of the #BlackL...
4,1000010,6/27/2020 23:59,b'RT @Hoosiers1986: Rational people have alway...


In [108]:
# Apply tweet preprocessing first using the tweet-preprocessing library.
# Define a preprocess function and use pandas to apply it on each value of 'tweet'
def preprocess_tweet(row):
    tweet = row['tweet']
    tweet = p.clean(tweet)
    return tweet

In [109]:
# Cleen tweets.
clean_twts['tweet'] = clean_twts.apply(preprocess_tweet, axis=1)

In [110]:
# Now we apply other manual tweet preprocessing
# stop word removal
def stopword_removal(row):
    tweet = row['tweet']
    tweet = remove_stopwords(tweet)
    return tweet

In [111]:
clean_twts['tweet'] = clean_twts.apply(stopword_removal, axis=1)
clean_twts.head()

,id,datetime,tweet
0,1000000,6/27/2020 23:59,"b""RT : Please help cold blooded killer gunned ..."
1,1000002,6/27/2020 23:59,"b""RT : Hi. It's over. Don't let over. """
2,1000006,6/27/2020 23:59,"b""RT : Stop treating communities therapy rage!..."
3,1000008,6/27/2020 23:59,b'RT : Only % protestors nationally actually b...
4,1000010,6/27/2020 23:59,b'RT : Rational people believed ALL LIVES MATT...


In [112]:
# Remove extra white spaces, punctuation and apply lower casing
clean_twts['tweet'] = clean_twts['tweet'].str.lower().str.replace('[^\w\s]', ' ').str.replace('\s\s+', ' ')
clean_twts.head()

,id,datetime,tweet
0,1000000,6/27/2020 23:59,b rt please help cold blooded killer gunned ki...
1,1000002,6/27/2020 23:59,b rt hi it s over don t let over
2,1000006,6/27/2020 23:59,b rt stop treating communities therapy rage n ...
3,1000008,6/27/2020 23:59,b rt only protestors nationally actually black...
4,1000010,6/27/2020 23:59,b rt rational people believed all lives matter...


In [113]:
# Remove duplicate tweets again post processing
clean_twts = clean_twts.drop_duplicates(subset='tweet', keep='first')
clean_twts.head()

,id,datetime,tweet
0,1000000,6/27/2020 23:59,b rt please help cold blooded killer gunned ki...
1,1000002,6/27/2020 23:59,b rt hi it s over don t let over
2,1000006,6/27/2020 23:59,b rt stop treating communities therapy rage n ...
3,1000008,6/27/2020 23:59,b rt only protestors nationally actually black...
4,1000010,6/27/2020 23:59,b rt rational people believed all lives matter...


In [114]:
clean_twts.count()

id          93638
datetime    93638
tweet       93638
dtype: int64

In [115]:
#exporting
clean_twts.to_csv ('clean_twts.csv', index = False, header=True)

In [116]:
clean_twts.head()

,id,datetime,tweet
0,1000000,6/27/2020 23:59,b rt please help cold blooded killer gunned ki...
1,1000002,6/27/2020 23:59,b rt hi it s over don t let over
2,1000006,6/27/2020 23:59,b rt stop treating communities therapy rage n ...
3,1000008,6/27/2020 23:59,b rt only protestors nationally actually black...
4,1000010,6/27/2020 23:59,b rt rational people believed all lives matter...


**<font color="blue">Step 3: Uplaoding the clean_twts.csv to a SQLite database</font>**  

**<font color="red">Code:</font>**

In [121]:
!pip install csv-to-sqlite

In [123]:
import csv_to_sqlite

In [138]:
options = csv_to_sqlite.CsvOptions(typing_style="full", encoding="windows-1250") 
input_files = ["clean_twts.csv"] # pass in the CSV file
csv_to_sqlite.write_csv(input_files, "clean_twts.sqlite", options)

Error on table clean_twts.csv: 
 table [clean_twts] already exists
Written 0 rows into 1 tables in 0.209 seconds


**<font color="blue">Step 4: Using tweets from SQLite database and passing them through IBM Watson Tone Analyzer </font>**  

**<font color="red">Code:</font>**

In [139]:
import sqlite3

In [140]:
conn = sqlite3.connect("clean_twts.sqlite")

In [141]:
# find out the table name in the database
def sql_fetch(conn):

    cursorObj = conn.cursor()

    cursorObj.execute('SELECT name from sqlite_master where type= "table"')

    print(cursorObj.fetchall())

sql_fetch(conn)

[('clean_twts',)]


In [142]:
#create a tweets dataframe from the database:
tweetsdf = pd.read_sql_query("SELECT * FROM clean_twts", conn)

In [136]:
tweetsdf.head()

,id,datetime,tweet
0,1000000,6/27/2020 23:59,b rt please help cold blooded killer gunned ki...
1,1000002,6/27/2020 23:59,b rt hi it s over don t let over
2,1000006,6/27/2020 23:59,b rt stop treating communities therapy rage n ...
3,1000008,6/27/2020 23:59,b rt only protestors nationally actually black...
4,1000010,6/27/2020 23:59,b rt rational people believed all lives matter...
